In [ ]:
from IPython.display import display, Markdown

scenario_info = """"""
scenario_folder = "scenario_02"

In [ ]:

display(Markdown("""
# Scenario Analysis - "Grid modelling to assess electrofuels supply potential - The impact of electrofuels on the US electricity grid"
"""))
display(Markdown(scenario_info))

---

## 1. Setup and Data Loading

*This section handles the initial setup, including importing necessary libraries and loading the solved PyPSA-Earth networks.*

### 1.1. Import Libraries

In [ ]:
!pip install plotly -qq

In [ ]:
import os
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use("Agg")

import matplotlib.pyplot as plt
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pycountry

import pypsa
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
from pathlib import Path

import cartopy.crs as ccrs
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
from matplotlib.offsetbox import AnnotationBbox, AuxTransformBox
from matplotlib.patches import Wedge
from matplotlib.patches import Patch
import matplotlib.path as mpath
import matplotlib.transforms as mtransforms
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerPatch
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.font_manager import FontProperties
from IPython.display import display, HTML, Markdown
from matplotlib.font_manager import FontProperties
from functools import reduce
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import LineString
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import yaml
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore")

from _helpers import *

In [ ]:
with open('plotting.yaml', 'r') as file:
    plotting = yaml.safe_load(file)

tech_colors = plotting['tech_colors']
nice_names = plotting['nice_names']
rename_techs_capex = plotting['rename_tech_capex']
rename_techs_opex = plotting['rename_tech_opex']
rename_tech_colors = plotting['renamed_tech_colors']
categories_capex = plotting['categories_capex']
categories_opex = plotting['categories_opex']
tech_power_color = plotting['tech_power_color']
nice_names_power = plotting['nice_names_power']
tech_power_order = plotting['tech_power_order']
emm_mapping = plotting['emm_mapping']

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
pd.set_option('display.width', None)    
pd.set_option('display.max_colwidth', None)

import plotly.io as pio
pio.renderers.default = 'colab'

### 1.2. Load Solved Network(s)

The `scenario_dir` variable can be used to switch to a different scenario. The base year results are common for all the scenarios.

In [ ]:
# -----------------------------
# Project folder path
# -----------------------------
project_root = Path().resolve()

# -----------------------------
# Base year network path
# -----------------------------
base_year_file = project_root / "results" / "base_year" / "elec_s_100_ec_lcopt_Co2L-3H_3H_2020_0.07_AB_0export.nc"

networks = {
    "Base_2023": pypsa.Network(base_year_file)
}

# -----------------------------
# Scenario folder path (single scenario)
# -----------------------------
scenario_dir = project_root / "results" / "scenarios" / f"{scenario_folder}" #change the number to choose the scenario of your interest

## Folder structure
#  project_root/
#  └── notebooks/
#       ├── scenario_analysis.ipynb
#       └── results/
#            ├── base_year/
#            │   ├── elec_s_100_ec_lcopt_Co2L-3H_3H_2020_0.071_AB_10export.nc
#            └── scenarios/
#                 ├── scenario_01/
#                 │   ├── elec_s_100_ec_lcopt_3H_3H_2030_0.071_AB_10export.nc
#                 │   ├── elec_s_100_ec_lcopt_3H_3H_2035_0.071_AB_10export.nc
#                 │   └── elec_s_100_ec_lcopt_3H_3H_2040_0.071_AB_10export.nc
#                 ├── scenario_02/
#                 ├── ...
#                 ├── scenario_10/

years = [2030, 2035, 2040]

# -----------------------------
# Upload available networks per year
# -----------------------------
for year in years:
    # Use lcopt only for scenarios 05 and 06, otherwise use lv1
    if scenario_folder in ["scenario_05", "scenario_06"]:
        file_name = f"elec_s_100_ec_lcopt_CCL-3H_3H_{year}_0.07_AB_0export.nc"
    else:
        file_name = f"elec_s_100_ec_lv1_CCL-3H_3H_{year}_0.07_AB_0export.nc"

    file_path = scenario_dir / file_name

    if file_path.exists():
        scenario_name = scenario_dir.name
        key = f"{scenario_name}_{year}"
        networks[key] = pypsa.Network(file_path)
        print(f"Loaded: {key}")
    else:
        print(f"Missing: {file_path.relative_to(project_root)}")

network_names = list(networks.keys())
print("\nUploaded networks:")
print(list(networks.keys()))

In [ ]:
grid_region_shapes_path = project_root / "data" / "needs_grid_regions_aggregated.geojson"
state_shapes_path = project_root / "data" / "gadm41_USA_1.json"

regions_onshore = gpd.read_file(grid_region_shapes_path)
emm_shapes_path = project_root / "data" / "EIA_market_module_regions" / "EMM_regions.geojson"
regional_fees = pd.read_csv("data/EIA_market_module_regions/regional_fees.csv")

In [ ]:
for network in networks.keys():
    attach_grid_region_to_buses(networks[network], grid_region_shapes_path)
    attach_state_to_buses(networks[network], state_shapes_path)
    attach_emm_region_to_buses(networks[network], emm_shapes_path, distance_crs="EPSG:3857")

---

## 2. Total system cost

In [ ]:
# SYSTEM COSTS (CAPEX + OPEX)

all_system_costs = []

for name_tag, network in networks.items():
    df_yearly = compute_system_costs(
        network=network,
        rename_capex=rename_techs_capex,
        rename_opex=rename_techs_opex,
        name_tag=name_tag
    )
    all_system_costs.append(df_yearly)

df_system_costs_all = pd.concat(all_system_costs, ignore_index=True)

# Clean technology labels (remove extra spaces, invisible chars)
df_system_costs_all["tech_label"] = (
    df_system_costs_all["tech_label"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.replace(r"\xa0", " ", regex=True)
    .str.strip()
)

# Assign macro-categories based on CAPEX and OPEX mappings
df_system_costs_all["macro_category"] = df_system_costs_all.apply(
    lambda row: assign_macro_category(row, categories_capex, categories_opex),
    axis=1
)

# --- unify OPEX categories so inputs appear in the plot ---
df_system_costs_all["cost_type"] = df_system_costs_all["cost_type"].replace(
    {"Operational expenditure (inputs)": "Operational expenditure"}
)

# PLOTS

# Capital expenditure by year
plot_stacked_costs_by_year_plotly(
    df_system_costs_all,
    "Capital expenditure",
    tech_colors=rename_tech_colors
)

# Operational expenditure by year (now includes inputs)
plot_stacked_costs_by_year_plotly(
    df_system_costs_all,
    "Operational expenditure",
    tech_colors=rename_tech_colors
)

showfig()


In [ ]:
try:
    del df_original_costs, power_techs, all_original_costs
except:
    pass

# 1. Reconstruction of costs (links only)
all_original_costs = []
for name_tag, network in networks.items():
    df_costs = compute_links_only_costs(network, name_tag)
    all_original_costs.append(df_costs)

df_original_costs = pd.concat(all_original_costs, ignore_index=True)

# 2. Add missing OPEX (generators and batteries)
extra_costs = []
for name_tag, network in networks.items():
    match = re.search(r"(20\d{2})", name_tag)
    scenario_year = int(match.group(1)) if match else None

    # Generators
    if hasattr(network, "generators"):
        for carrier, subdf in network.generators.groupby("carrier"):
            # Exclude fossil and nuclear technologies, already covered in links
            if carrier.lower() in ["coal", "oil", "biomass", "gas", "nuclear"]:
                continue

            opex = (subdf.marginal_cost * subdf.p_nom_opt * 8760).sum() / 1e9
            if opex > 0:
                extra_costs.append({
                    "tech_label": carrier,
                    "cost_type": "Operational expenditure",
                    "cost_billion": opex,
                    "year": scenario_year,
                    "scenario": name_tag
                })

    # Batteries (stores)
    if hasattr(network, "stores"):
        for carrier, subdf in network.stores.groupby("carrier"):
            if "battery" not in carrier.lower():
                continue

            opex = (subdf.marginal_cost * subdf.e_nom_opt * 8760).sum() / 1e9
            if opex > 0:
                extra_costs.append({
                    "tech_label": carrier,
                    "cost_type": "Operational expenditure",
                    "cost_billion": opex,
                    "year": scenario_year,
                    "scenario": name_tag
                })

# Concatenate any new OPEX data
if extra_costs:
    df_extra_opex = pd.DataFrame(extra_costs)
    df_original_costs = pd.concat([df_original_costs, df_extra_opex], ignore_index=True)

# --- unify OPEX types so input-based costs are included ---
df_original_costs["cost_type"] = df_original_costs["cost_type"].replace(
    {"Operational expenditure (inputs)": "Operational expenditure"}
)

# 3. Normalize technology names to match YAML mapping
df_original_costs["tech_label"] = df_original_costs["tech_label"].replace({
    "onwind": "Onshore Wind",
    "offwind-ac": "Offshore Wind (AC)",
    "offwind-dc": "Offshore Wind (DC)",
    "solar": "Solar",
    "ror": "Run of River",
    "geothermal": "Geothermal",
    "nuclear": "Nuclear"
})

# 4. Process data for visualization
power_techs = identify_power_generation_technologies(
    rename_techs_capex, rename_techs_opex, categories_capex, categories_opex
)

mask = df_original_costs["tech_label"].isin(nice_names_power.keys())
df_power = df_original_costs[mask].copy()
df_power["tech_label"] = df_power["tech_label"].replace(nice_names_power)

df_power_aggregated = df_power.groupby(
    ['tech_label', 'cost_type', 'year', 'scenario'], 
    as_index=False
).agg({'cost_billion': 'sum'})

power_techs_mapped = set(df_power_aggregated['tech_label'].unique())

# 5. Create plots
fig1 = plot_power_generation_details(
    df_power_aggregated,
    "Capital expenditure",
    power_techs_mapped,
    tech_colors=tech_power_color,
    nice_names=None,
    tech_order=tech_power_order
)

fig2 = plot_power_generation_details(
    df_power_aggregated,
    "Operational expenditure", 
    power_techs_mapped,
    tech_colors=tech_power_color,
    nice_names=None,
    tech_order=tech_power_order
)

fig1.show()
fig2.show()


In [ ]:
total_tax_credit_table = {}
for name, network in networks.items():
    tax_credit_df = compute_power_opex_with_tax_credits(network, name)
    tax_credit_df.tech_label = tax_credit_df["tech_label"].replace(nice_names)
    tax_credit_df.tech_label = tax_credit_df["tech_label"].replace(nice_names_power)
    total_tax_credit_table[name] = tax_credit_df

total_tax_credit_df = pd.concat(total_tax_credit_table, ignore_index=True)

In [ ]:
techs_to_show = ["Nuclear", "Geothermal", "Biomass", "Biomass CHP", "Utility-scale solar", "Onshore wind", "Offshore wind"]
fig = plot_tax_credit_cluster_bars(total_tax_credit_df.query("tech_label in @techs_to_show"), 
                                   tech_power_color, nice_names_power, 
                                   title="Power Generation: OPEX With vs. Without Tax Credits",
                                   cost_type="OPEX")
fig.show()

In [ ]:
for network_name, net in networks.items():

    # Extract last block of 4-digit year
    match = re.findall(r"\d{4}", network_name)
    year = match[-1] if match else network_name

    print(f"\nYear: {year}\n")

    co2_cap = compute_captured_co2_by_tech_from_network(net)
    df = compute_cc_energy_costs(net, year, co2_cap)
    display_cc_summary(df)

In [ ]:
electrolyzer_carriers = {
    "Alkaline electrolyzer large",
    "PEM electrolyzer",
    "SOEC"
}

fig = plot_tax_credit_cluster_bars(total_tax_credit_df.query("tech_label in @electrolyzer_carriers"), 
                                   tech_colors, rename_techs_opex, 
                                   title="Hydrogen production OPEX With vs. Without Tax Credits",
                                   unit="million", cost_type="OPEX")
fig.show()

In [ ]:
total_capex_tax_credit_table = {}

for name, network in networks.items():
    capex_tax_credit_df = compute_power_capex_with_tax_credits(
        network, name
    )
    total_capex_tax_credit_table[name] = capex_tax_credit_df

total_capex_tax_credit_df = pd.concat(
    total_capex_tax_credit_table.values(),
    ignore_index=True
)

plot_tax_credit_cluster_bars(
    total_capex_tax_credit_df.query("tech_label == 'battery'"),
    tech_colors,
    rename_techs_opex,
    title="Battery: CAPEX With vs. Without Tax Credits",
    unit="million",
    cost_type="CAPEX",
)

In [ ]:
df_h2_efuels_results = create_h2_efuels_analysis(networks)

## 3. Electricity demand

*This section illustrates results concerning electricity demand across the different sectors in the model, with a specific focus on data centers.*

### 3.1. Electricity demand by service
*This section reports results for electricity demand related to the different sectors, providing a comparison against the NREL Electrification Futures Studies (EFS) scenarios.*

In [ ]:
# Industrial processes consuming AC electricity
target_processes = [
    "SMR CC", "Haber-Bosch", "ethanol from starch", "ethanol from starch CC",
    "DRI", "DRI CC", "DRI H2", "BF-BOF", "BF-BOF CC", "EAF",
    "dry clinker", "cement finishing", "dry clinker CC"
]

# Static and dynamic loads
static_load_carriers = ["rail transport electricity", "agriculture electricity", "industry electricity"]
dynamic_load_carriers = ["AC", "services electricity", "land transport EV", "data center"]

demand = pd.DataFrame(columns=networks.keys(),
                      index=dynamic_load_carriers + static_load_carriers + ["total demand"])
demand.index.name = "Load type (TWh)"

for name, n in networks.items():
    nhours = n.snapshot_weightings.objective.sum()

    # Static loads
    static_totals = (
        n.loads.groupby("carrier").sum().p_set
        .reindex(static_load_carriers)
        .fillna(0)
    )
    static_load_twh = static_totals.sum() * nhours / 1e6

    # Industrial AC
    process_links = n.links[n.links.carrier.isin(target_processes)]
    ac_input_links = process_links[process_links.bus0.map(n.buses.carrier) == "AC"].index
    ind_ac_profile = n.links_t.p0[ac_input_links].sum(axis=1)
    ind_ac_twh = (ind_ac_profile * n.snapshot_weightings.objective).sum() / 1e6

    # Non-industrial AC
    ac_loads = n.loads[n.loads.carrier == "AC"]
    industrial_ac_buses = n.links.loc[ac_input_links, "bus0"].unique()
    ac_non_ind_idx = ac_loads[~ac_loads.bus.isin(industrial_ac_buses)].index
    ac_profile = n.loads_t.p_set[ac_non_ind_idx.intersection(n.loads_t.p_set.columns)].sum(axis=1)
    ac_twh = (ac_profile * n.snapshot_weightings.objective).sum() / 1e6 - ind_ac_twh

    # Services and EVs
    serv_idx = [i for i in n.loads[n.loads.carrier == "services electricity"].index
                if i in n.loads_t.p_set.columns]
    ev_idx = [i for i in n.loads[n.loads.carrier == "land transport EV"].index
              if i in n.loads_t.p_set.columns]
    serv_profile = n.loads_t.p_set[serv_idx].sum(axis=1) if serv_idx else 0
    ev_profile = n.loads_t.p_set[ev_idx].sum(axis=1) if ev_idx else 0
    serv_twh = (serv_profile * n.snapshot_weightings.objective).sum() / 1e6
    ev_twh = (ev_profile * n.snapshot_weightings.objective).sum() / 1e6

    # Data centers
    data_center_p_set_sum = n.loads.loc[n.loads.carrier == "data center", "p_set"].sum()
    data_center_twh = data_center_p_set_sum * nhours / 1e6

    # Other electricity (included in industrial consumption)
    other_idx = [i for i in n.loads[n.loads.carrier == "other electricity"].index
                 if i in n.loads_t.p_set.columns]
    other_profile = n.loads_t.p_set[other_idx].sum(axis=1) if other_idx else 0
    other_twh = (other_profile * n.snapshot_weightings.objective).sum() / 1e6

    industry_static_twh = static_totals.get("industry electricity", 0) * nhours / 1e6
    industry_elec_twh = industry_static_twh + ind_ac_twh + other_twh

    demand.loc["rail transport electricity", name] = static_totals.get("rail transport electricity", 0) * nhours / 1e6
    demand.loc["agriculture electricity", name] = static_totals.get("agriculture electricity", 0) * nhours / 1e6
    demand.loc["industry electricity", name] = industry_elec_twh
    demand.loc["AC", name] = ac_twh
    demand.loc["services electricity", name] = serv_twh
    demand.loc["land transport EV", name] = ev_twh
    demand.loc["data center", name] = data_center_twh
    demand.loc["total demand", name] = (
        static_load_twh + ac_twh + ind_ac_twh + serv_twh + ev_twh + data_center_twh + other_twh
    )
    demand.loc["total demand no data center", name] = (
        static_load_twh + ac_twh + ind_ac_twh + serv_twh + ev_twh + other_twh
    )


In [ ]:
# NREL reference values (2023, 2030, 2040)
nrel_data_medium = {
    "AC (residential + some industrial processes)": [1360.81, 1359.91, 1392.19],
    "services electricity": [1381.11, 1398.69, 1579.71],
    "land transport EV":    [13.10,	275.97,	714.41],
    "rail transport electricity": [7.29, 7.64, 8.29],
    "agriculture electricity": [np.nan, np.nan, np.nan],
    "industry electricity": [1036.81, 1109.69, 1213.94],
    "data center": [np.nan, np.nan, np.nan],
    "total demand (incl. data centers)": [3799.12, 4293.30, 5072.86],
    "total demand (excl. data centers)": [3799.12, 4293.30, 5072.86],
}

nrel_data_high = {
    "AC (residential + some industrial processes)": [1360.81, 1395.21, 1529.51],
    "services electricity": [1381.11, 1470.10, 1673.71],
    "land transport EV":    [13.10, 349.15, 1039.76],
    "rail transport electricity": [7.29, 9.96, 12.71],
    "agriculture electricity": [np.nan, np.nan, np.nan],
    "industry electricity": [1036.81, 1152.90, 1286.24],
    "data center": [np.nan, np.nan, np.nan],
    "total demand (incl. data centers)": [3799.12, 4377.32,	6713.68],
    "total demand (excl. data centers)": [3799.12, 4377.32,	6713.68],
}

# Mapping PyPSA names to NREL names
row_rename_map = {
    "AC": "AC (residential + some industrial processes)",
    "total demand": "total demand (incl. data centers)",
    "total demand no data center": "total demand (excl. data centers)"
}

# Extract years from network names
network_years = {}
for key in networks:
    match = re.search(r'(\d{4})', key)
    if match:
        network_years[int(match.group(1))] = key

# Determine the current scenario from network names (e.g., scenario_05)
scenario_keys = [k for k in networks if "scenario_" in k]
if scenario_keys:
    current_scenario = scenario_keys[0].split("_")[0] + "_" + scenario_keys[0].split("_")[1]
else:
    current_scenario = None

# Select appropriate NREL reference data based on scenario
if current_scenario in ["scenario_05", "scenario_06"]:
    nrel_data = nrel_data_high
else:
    nrel_data = nrel_data_medium

# Prepare the comparison table
rows = list(nrel_data.keys())
columns = pd.MultiIndex.from_product(
    [sorted(network_years.keys()), ["PyPSA-Earth", "NREL", "diff %"]]
)
comparison_df = pd.DataFrame(index=rows, columns=columns)

# Populate the comparison values
for year, net_key in network_years.items():
    for row in rows:
        pypsa_row = {v: k for k, v in row_rename_map.items()}.get(row, row)
        
        pypsa_val = demand.at[pypsa_row, net_key] if pypsa_row in demand.index else np.nan
        nrel_vals = nrel_data.get(row, None)

        if nrel_vals and year in [2023, 2030, 2040]:
            col_idx = [2023, 2030, 2040].index(year)
            nrel_val = nrel_vals[col_idx]
        else:
            nrel_val = np.nan

        comparison_df.at[row, (year, "PyPSA-Earth")] = pypsa_val
        comparison_df.at[row, (year, "NREL")] = nrel_val

        if pd.notna(pypsa_val) and pd.notna(nrel_val) and nrel_val != 0:
            diff = 100 * (pypsa_val - nrel_val) / nrel_val
        else:
            diff = np.nan
        comparison_df.at[row, (year, "diff %")] = diff

# Style function to highlight significant differences
def highlight_diff(val):
    if pd.isna(val):
        return ''
    if val > 10:
        return 'background-color: #ffcccc'  # light red
    elif val < -10:
        return 'background-color: #cce5ff'  # light blue
    else:
        return 'background-color: #d4edda'  # light green

# Apply styling to the diff columns
styled = comparison_df.style
for year in network_years:
    styled = styled.applymap(highlight_diff, subset=pd.IndexSlice[:, (year, "diff %")])
styled = styled.format(na_rep="N/A", precision=2)

# Display final table
print("\n Electric load by sector (TWh/year)")
display(styled)

In [ ]:
# Extract years from network names
years = {}
for name in networks:
    match = re.search(r"\d{4}", name)
    if match:
        years[int(match.group())] = name
years = dict(sorted(years.items()))  # sort by year

# Compute y-axis max (in GW) and build demand profiles
y_max = 0
demand_profiles_incl = {}
demand_profiles_excl = {}

for year, name in years.items():
    n = networks[name]
    nhours = n.snapshot_weightings.objective.sum()

    # Dynamic loads: AC, services, EV, other electricity
    dynamic_loads = n.loads[n.loads.carrier.isin(["AC", "services electricity", "land transport EV", "other electricity"])]
    dynamic_idx = dynamic_loads.index.intersection(n.loads_t.p_set.columns)
    dyn_profile = n.loads_t.p_set[dynamic_idx].sum(axis=1)

    # Static loads: rail, agriculture, industry
    static_carriers = ["rail transport electricity", "agriculture electricity", "industry electricity"]
    static_load = n.loads[n.loads.carrier.isin(static_carriers)]
    static_sum = static_load.groupby("carrier").sum()["p_set"].sum()  # MW
    static_profile = pd.Series(static_sum, index=n.snapshots)

    # Data centers (flat profile if present)
    if "data center" in n.loads.carrier.values:
        dc_sum = n.loads[n.loads.carrier == "data center"]["p_set"].sum()
        dc_profile = pd.Series(dc_sum, index=n.snapshots)
    else:
        dc_profile = 0

    # Total profiles (in GW)
    profile_excl = (dyn_profile + static_profile) / 1000
    profile_incl = (dyn_profile + static_profile + dc_profile) / 1000

    demand_profiles_excl[year] = profile_excl
    demand_profiles_incl[year] = profile_incl

    y_max = max(y_max, profile_incl.max())

y_max *= 1.05  # add 5% margin

# Plot profiles for each year
fig, axes = plt.subplots(nrows=len(years), figsize=(12, 4 * len(years)), sharey=True)

if len(years) == 1:
    axes = [axes]  # ensure list if only one subplot

for ax, (year, name) in zip(axes, years.items()):
    demand_profiles_excl[year].plot(ax=ax, linewidth=0.5, color="tab:blue", label="Excl. data centers")
    demand_profiles_incl[year].plot(ax=ax, linewidth=0.5, color="tab:orange", label="Incl. data centers")

    ax.set_title(f"\nTotal electricity demand profile - {year}\n")
    ax.set_xlabel("Time (-)")
    ax.set_ylabel("Demand [GW]")
    ax.set_ylim(0, y_max)
    ax.legend(loc="upper right")

plt.tight_layout()
showfig()


### 3.2. Data center loads
*This section shows the subdivision of data center loads across the different States / Grid Regions.*

In [ ]:
data_center_load = {}
for key, net in networks.items():
    year_match = re.search(r"\d{4}", key)
    short_key = year_match.group() if year_match else key 
    data_center_load[short_key] = compute_data_center_load(net)
    
# Compute the maximum value to normalize the y-axis
max_val = 0
for df in data_center_load.values():
    demand_by_grid_region = df.groupby("grid_region")["p_set"].sum().div(1e3)
    demand_by_grid_region = demand_by_grid_region[demand_by_grid_region > 0]
    if not demand_by_grid_region.empty:
        max_val = max(max_val, demand_by_grid_region.max())

ymax = max(max_val * 1.1, 1)

# Main loop to generate plots
for year, df in data_center_load.items():
    demand_by_grid_region = df.groupby("grid_region")["p_set"].sum().div(1e3)
    demand_by_grid_region = demand_by_grid_region[demand_by_grid_region > 0]

    if demand_by_grid_region.empty:
        print(f"\nSkipping {year}: No data centers demand\n")
        continue

    ax = demand_by_grid_region.plot(
        kind="bar",
        title=f"Data center electricity demand by Grid region - {year}",
        ylabel="Data centers demand (GW)",
        xlabel="Grid region",
        figsize=(8, 4),
        legend=False
    )

    # Add text labels on top of bars
    for i, value in enumerate(demand_by_grid_region.values):
        ax.text(
            i,
            value + ymax * 0.01,  # Slightly above the bar
            f"{value:.1f}",       # Format with 1 decimal place
            ha="center",
            va="bottom",
            fontsize=9,
            rotation=0
        )

    ax.set_ylim(0, ymax)
    plt.xticks(rotation=-30)
    plt.tight_layout()
    showfig()

In [ ]:
for key, df in data_center_load.items():    
    year_match = re.search(r"\d{4}", key)
    short_key = year_match.group() if year_match else key 
    demand_by_state = df.groupby("state")["p_set"].sum().div(1e3)
    demand_by_state = demand_by_state[demand_by_state > 0]

    if demand_by_state.empty:
        print(f"\nSkipping {key}: No data centers demand\n")
        continue

    # Extract year from key assuming format like "scenario_01_2030"
    year_match = re.search(r"\d{4}", key)
    year = year_match.group() if year_match else "Unknown Year"

    ax = demand_by_state.plot(
        kind="bar",
        title=f"Data center electricity demand by State - {year}",
        ylabel="Data centers demand (GW)",
        xlabel="State",
        figsize=(11, 4),
        legend=False
    )

    # Add text labels on top of bars
    for i, value in enumerate(demand_by_state.values):
        ax.text(
            i,
            value + max_val * 0.01,  # Slightly above the bar
            f"{value:.1f}",
            ha="center",
            va="bottom",
            fontsize=9
        )

    ax.set_ylim(0, max_val * 1.05)
    plt.xticks(rotation=0) 
    plt.tight_layout()
    showfig()

### 3.3 Renewable Potential

In [ ]:
fig1, ax1, data1 = plot_solar_cf_from_network(networks["Base_2023"], plot_type='scatter')
showfig()

In [ ]:
fig1, ax1, data1 = plot_solar_cf_from_network(networks["Base_2023"], plot_type='weighted')
showfig()

In [ ]:
fig1, ax1, data1 = plot_wind_cf_from_network(networks["Base_2023"], plot_type='scatter')
showfig()

In [ ]:
fig1, ax1, data1 = plot_wind_cf_from_network(networks["Base_2023"], plot_type='weighted')
showfig()

---

## 4. Installed capacity

*Here, we analyze the optimal installed capacities of generation, storage, and conversion technologies as determined by the model.*

### 4.1. Installed electricity capacity
*A map showing the total installed capacity (in GW) for each electricity generation carrier (e.g., Solar, Onshore Wind, Offshore Wind) at different locations.*

In [ ]:
# regions_onshore = gpd.read_file(grid_region_shapes_path)

# # Prepare Data for Plotting
# gen_p_nom_opt = n.generators.groupby(["bus", "carrier"]).p_nom_opt.sum()
# sto_p_nom_opt = n.storage_units.groupby(["bus", "carrier"]).p_nom_opt.sum()
# bus_carrier_capacity = pd.concat([gen_p_nom_opt, sto_p_nom_opt])
# bus_carrier_capacity = bus_carrier_capacity[bus_carrier_capacity > 0]

# line_values = n.lines.s_nom
# links_values = n.links[n.links.carrier == "DC"].p_nom

# n = update_ac_dc_bus_coordinates(n)

# # Find carriers that need a color
# carriers_to_plot = bus_carrier_capacity.index.get_level_values(1).unique()
# for carrier in carriers_to_plot:
#     if carrier not in n.carriers.index:
#         n.add("Carrier", carrier)

# for idx in tech_colors.keys():
#     n.carriers.loc[idx, "color"] = tech_colors[idx]
    
# fill_missing_nice_names(n, nice_names)

# # Assign colors from the dictionary, using 'gray' for any not found
# n.carriers["color"] = n.carriers.index.map(tech_colors)
# n.carriers.color.fillna("gray", inplace=True)

In [ ]:
for key, n in networks.items():
    plot_network_generation_and_transmission(n, key, tech_colors, nice_names, regions_onshore)

In [ ]:
df_capacity = compute_installed_capacity_by_carrier(networks, nice_names=nice_names)

In [ ]:
# Invert nice_names so we can map pretty name to original PyPSA carrier
inverse_nice = {v: k for k, v in nice_names.items()}

def color_for_pretty(pretty_name: str) -> str:
    """Get the color for a pretty name, using tech_colors based on the original key"""
    orig = inverse_nice.get(pretty_name, pretty_name)
    return tech_colors.get(orig, "gray")

# Compute installed capacities (GW) per carrier
carrier_capacity_df = compute_installed_capacity_by_carrier(networks, display_result=False)

# Preferred order (matching the summary table)
preferred_order = [
    "Coal", "Gas CCGT", "Gas OCGT", "Gas CHP",
    "Oil", "Nuclear",
    "Biomass", "Biomass CHP", "Geothermal",        
    "Conventional hydro", "Run-of-River hydro", "Pumped hydro storage",
    "Onshore wind", "Offshore wind",
    "Utility-scale solar", "Rooftop solar", "CSP",
    "Battery"
]

# Convert pretty names → original carrier keys, keep only available ones
available = carrier_capacity_df.index.tolist()
preferred_index = [
    inverse_nice.get(p, p) for p in preferred_order if inverse_nice.get(p, p) in available
]
rest = [c for c in available if c not in preferred_index]
ordered_index = preferred_index + rest

# Reorder DataFrame rows
carrier_capacity_df = carrier_capacity_df.loc[ordered_index]

# Colors in the same order
color_list = [tech_colors.get(carrier, 'gray') for carrier in carrier_capacity_df.index]

# Create stacked bar chart with Plotly
fig = go.Figure()

for carrier, color in zip(carrier_capacity_df.index, color_list):
    pretty_name = nice_names.get(carrier, carrier)  # show nice names in legend/hover
    fig.add_trace(go.Bar(
        name=pretty_name,
        x=carrier_capacity_df.columns.astype(str),
        y=carrier_capacity_df.loc[carrier],
        marker_color=color,
        width=0.6,
        hovertemplate=f"%{{x}}<br>{pretty_name}: %{{y:.2f}} GW <extra></extra>"
    ))

# Layout settings
fig.update_layout(
    barmode='stack',
    legend_title="Technology",
    legend_traceorder="reversed",  # <-- reversed order for easier readability
    title=dict(
        text="Installed capacity - Electricity generation",
        x=0.5, xanchor='center',
        font=dict(size=16)
    ),
    xaxis_title="Year (-)",
    yaxis_title="Installed capacity (GW)",
    template="plotly_white",
    width=1200,
    height=800,
    xaxis=dict(
        tickangle=0,
        showline=True,
        linecolor='black'
    ),
    yaxis=dict(
        showline=True,
        linecolor='black'
    ),
    font=dict(size=12),
)

# Optional border around the plot
fig.add_shape(
    type="rect",
    xref="paper", yref="paper",
    x0=0, y0=0, x1=1, y1=1,
    line=dict(color="black", width=1),
    layer="below"
)

fig.show()


### 4.2. Transmission Line Expansion

In [ ]:
for network_name, n in networks.items():
    year_match = re.search(r'\d{4}', network_name)
    year = year_match.group() if year_match else network_name
    
    df = compute_line_expansion_capacity(n)[0]  # grouped by grid_region
    
    df.index.name = 'Grid Region'
    
    df = df.rename(columns={
        "s_nom": "Transmission capacity (GW) - 2023",
        "s_nom_opt": f"Expanded transmission capacity (GW) - {year}"
    })
   
    df_display = df.style.format("{:.2f}")
    
    display(df_display)

In [ ]:
for network_name, n in networks.items():
    year_match = re.search(r'\d{4}', network_name)
    year = year_match.group() if year_match else network_name
    
    df = compute_line_expansion_capacity(n)[1]  # grouped by state
    
    df.index.name = 'State'
    
    df = df.rename(columns={
        "s_nom": "Transmission capacity (GW) - 2023",
        "s_nom_opt": f"Expanded transmission capacity (GW) - {year}"
    })
    
    df_display = df.style.format("{:.2f}")
    
    display(df_display)

In [ ]:
for network_name, n in networks.items():
    line_expan_cap = compute_line_expansion_capacity(n)[0]

    line_expan_cap = line_expan_cap.rename(columns={
        "s_nom": "Installed (2023)",
        "s_nom_opt": "Expanded"
    })
    
    year = network_name.split('_')[-1]
    line_expan_cap.plot(
        kind="bar",
        title=f"Transmission capacity by Grid Region - {year}",
        ylabel="Capacity (GW)",
        xlabel="Grid regions",
        figsize=(10, 5),
        legend=True
    )

    plt.xticks(rotation=45)
    plt.tight_layout()
    showfig()

In [ ]:
for network_name, n in networks.items():
    line_expan_cap = compute_line_expansion_capacity(n)[1]

    line_expan_cap = line_expan_cap.rename(columns={
        "s_nom": "Installed (2023)",
        "s_nom_opt": "Expanded"
    })
    
    year = network_name.split('_')[-1]
    line_expan_cap.plot(
        kind="bar",
        title=f"Transmission capacity by State - {year}",
        ylabel="Capacity (GW)",
        xlabel="States",
        figsize=(18, 6),
        legend=True
    )

    plt.xticks(rotation=45)
    plt.tight_layout()
    showfig()

### 4.3. Hydrogen production capacity
*This section reports results about installed hydrogen production capacity of the different electrolyzer technologies (e.g., Alkaline, PEM, SOEC).*

In [ ]:
for key, net in networks.items():
    year = key[-4:]
    plot_h2_capacities_map(net, year, tech_colors, nice_names, regions_onshore)

In [ ]:
h2_cap_by_network = {}
max_n_states = 0
max_capacity = 0

for name, network in networks.items():
    h2_cap = compute_h2_capacities(network).groupby('state')[
        ['Alkaline electrolyzer large', 'PEM electrolyzer', 'SOEC']
    ].sum()

    filtered = h2_cap[h2_cap.sum(axis=1) > 1]

    h2_cap_by_network[name] = filtered

    if not filtered.empty:
        max_n_states = max(max_n_states, filtered.shape[0])
        max_capacity = max(max_capacity, filtered.sum(axis=1).max())

for name, filtered in h2_cap_by_network.items():
    year = name[-4:]

    if filtered.empty:
        print(f"\nSkipping {year}: no States with capacity > 1 MW\n")
        continue

    plot_h2_capacities_by_state(filtered, year, max_capacity, max_n_states)


In [ ]:
h2_cap_by_network = {}
max_n_grid_regions = 0
max_capacity = 0

for name, network in networks.items():
    h2_cap = compute_h2_capacities(network).groupby('grid_region')[
        ['Alkaline electrolyzer large', 'PEM electrolyzer', 'SOEC']
    ].sum()

    filtered = h2_cap[h2_cap.sum(axis=1) > 1]

    h2_cap_by_network[name] = filtered

    if not filtered.empty:
        max_n_grid_regions = max(max_n_grid_regions, filtered.shape[0])
        max_capacity = max(max_capacity, filtered.sum(axis=1).max())

for name, filtered in h2_cap_by_network.items():
    year = name[-4:]

    if filtered.empty:
        print(f"\nSkipping {year}: no Grid Regions with capacity > 1 MW\n")
        continue

    plot_h2_capacities_by_grid_region(filtered, year, max_capacity, max_n_grid_regions)


In [ ]:
for idx, network in enumerate(networks.keys()):
    year = network[-4:]
    
    h2_state_cap = compute_h2_capacities(networks[network]).groupby(['state'])[
        ['Alkaline electrolyzer large', 'PEM electrolyzer', 'SOEC']
    ].sum()

    print(f"\nInstalled electrolyzer capacity by State (MW input electricity) - {year} (only States with capacity ≥ 1 MW)\n")

    df = h2_state_cap.T
    df = df.loc[:, (df >= 1).any(axis=0)]

    df.columns.name = "State"
    df.index.name = None

    display(df.style.format("{:.1e}"))

In [ ]:
for idx, network in enumerate(networks.keys()):
    year = network[-4:]
    h2_state_cap = compute_h2_capacities(networks[network]).groupby(['grid_region'])[
        ['Alkaline electrolyzer large', 'PEM electrolyzer', 'SOEC']
    ].sum()

    print(f"\nInstalled electrolyzer capacity (MW input electricity) by Grid Region - {year} (only Grid Regions with capacity ≥ 1 MW))\n")

    df = h2_state_cap.sort_values(by=list(h2_state_cap.columns), ascending=False).T
    df.columns.name = "Grid Region"
    df.index.name = None
    df = df.loc[:, (df >= 1).any(axis=0)]

    display(df.style.format("{:.1e}"))

### 4.4. e-kerosene production capacity
*This sector reports information about installed capacity for Fischer-Tropsch synthesis plants for e-kerosene production.*

In [ ]:
ft_state_table = {}

for name, net in networks.items():
    fig, ax, data = create_ft_capacity_by_state_map(
        net.copy(),
        path_shapes=state_shapes_path,
        network_name=name,
        min_capacity_gw=0.4
    )
    ax.set_aspect('equal', adjustable='datalim')
    showfig()

    
    # data['grid_region'] = data.bus0.map(n.buses.grid_region)
    ft_state_capacity = data.groupby('state')['p_nom_gw'].sum()
    ft_state_table[name] = ft_state_capacity

In [ ]:
# Representative plant capacity in MW (can be changed easily)
plant_capacity_mw = 400  

df = pd.DataFrame(ft_state_table)
df = df.reset_index()
df.rename(columns={df.columns[0]: 'State'}, inplace=True)
df = df.set_index('State')

# Extract only the year from network names
df.columns = [
    re.search(r'\d{4}', c).group() if re.search(r'\d{4}', c) else c
    for c in df.columns
]

df = df.fillna(0)

# Work with numeric values for filtering and plant calculation
df_num = df.astype(float)

# Apply filter year by year: set values < 0.4 GW to 0
df_num = df_num.where(df_num >= 0.4, 0)

# Compute plants (floor division by representative plant capacity)
df_plants = np.floor(df_num * 1000 / plant_capacity_mw).astype(int)

# Build combined DataFrame with labels like "Capacity (...) - 2025" and "N. of plants - 2025"
df_combined = pd.DataFrame(index=df_num.index)
for year in df_num.columns:
    df_combined[f"Capacity (GW input H2) - {year}"] = df_num[year].apply(lambda x: f"{x:.3e}")
    df_combined[f"N. of plants - {year}"] = df_plants[year]

# Style: highlight cells where plants >= 1 (both capacity and plants)
def highlight_df(df):
    styles = pd.DataFrame("", index=df.index, columns=df.columns)
    for col in df.columns:
        if "N. of plants" in col:
            styles[col] = df[col].apply(
                lambda v: "background-color: #fff2b2;" if isinstance(v, (int, float)) and v >= 1 else ""
            )
        if "Capacity" in col:
            year = col.split(" - ")[1]
            plant_col = f"N. of plants - {year}"
            styles[col] = df[plant_col].apply(
                lambda v: "background-color: #fff2b2;" if v >= 1 else ""
            )
    return styles

styled_df = df_combined.style.apply(highlight_df, axis=None)

display(Markdown(
    f"\nInstalled FT capacity by State (GW input H2) "
    f"and equivalent number of representative plants ({plant_capacity_mw} MW each)\n"
))
display(styled_df)


In [ ]:
ft_grid_table = {}

for name, net in networks.items():
    fig, ax, data = create_ft_capacity_by_grid_region_map(
        net.copy(),
        path_shapes=grid_region_shapes_path,
        network_name=name,
        min_capacity_gw=0.4
    )
    ax.set_aspect('equal', adjustable='datalim')
    showfig()

    ft_grid_table[name] = data.set_index('grid_region')['total_gw']

In [ ]:
# Representative plant capacity in MW (can be changed easily)
plant_capacity_mw = 400  

ft_grid_table = {
    k: v.groupby(level=0).sum() if v.index.duplicated().any() else v
    for k, v in ft_grid_table.items()
}

# Combine all scenario Series safely, even if grid regions differ
df = pd.concat(ft_grid_table, axis=1)

df = df.reset_index()
df.rename(columns={df.columns[0]: 'Grid Region'}, inplace=True)
df = df.set_index('Grid Region')

# Extract only the year from network names
df.columns = [
    re.search(r'\d{4}', c).group() if re.search(r'\d{4}', c) else c 
    for c in df.columns
]

df = df.fillna(0)

# Work with numeric values for filtering and plant calculation
df_num = df.astype(float)

# Apply filter year by year: set values < 0.4 GW to 0
df_num = df_num.where(df_num >= 0.4, 0)

# Compute plants (floor division by representative plant capacity)
df_plants = np.floor(df_num * 1000 / plant_capacity_mw).astype(int)

# Build combined DataFrame with labels like "Capacity (...) - 2025" and "N. of plants - 2025"
df_combined = pd.DataFrame(index=df_num.index)
for year in df_num.columns:
    df_combined[f"Capacity (GW input H2) - {year}"] = df_num[year].apply(lambda x: f"{x:.3e}")
    df_combined[f"N. of plants - {year}"] = df_plants[year]

# Style: highlight cells where plants >= 1 (both capacity and plants)
def highlight_df(df):
    styles = pd.DataFrame("", index=df.index, columns=df.columns)
    for col in df.columns:
        if "N. of plants" in col:
            styles[col] = df[col].apply(
                lambda v: "background-color: #fff2b2;" if isinstance(v, (int, float)) and v >= 1 else ""
            )
        if "Capacity" in col:
            year = col.split(" - ")[1]
            plant_col = f"N. of plants - {year}"
            styles[col] = df[plant_col].apply(
                lambda v: "background-color: #fff2b2;" if v >= 1 else ""
            )
    return styles

styled_df = df_combined.style.apply(highlight_df, axis=None)

display(Markdown(
    f"\nInstalled FT capacity by Grid Region (GW input H2) "
    f"and equivalent number of representative plants ({plant_capacity_mw} MW each)\n"
))
display(styled_df)


---

##  5. Operational analysis

*This section explores electricity and hydrogen generation and dispatch.*

### 5.1. Electricity dispatch
*Electricity dispatch is assessed for each single source across the model years.*

In [ ]:
elec_dispatch_data = plot_electricity_dispatch(
    networks, 
    tech_colors, 
    nice_names, 
    title_year=True, 
    return_data=False
)

### 5.2. Electricity generation
*Total electricity generation by source is reported by year, with an indication of the percentage covered by renewable or clean (renewables + nuclear) energy sources per State and at country-wise level.*

In [ ]:
generation_summary = {}

for key, network in networks.items():
    year = key[-4:]  # Extract the year
    generation_summary[year] = calculate_total_generation_by_carrier(network)

generation_df = pd.DataFrame.from_dict(generation_summary, orient='index').fillna(0)
generation_df = generation_df.sort_index().round(2)

generation_df.columns = [nice_names.get(c, c) for c in generation_df.columns]

preferred_order = [
    "Coal", "Oil", "Gas OCGT", "Gas CCGT", "Gas CHP", "Nuclear", "Biomass", "Biomass CHP", "Conventional hydro", "Run-of-River hydro", "Geothermal", "Utility-scale solar", "Rooftop solar", "Onshore wind", "Offshore wind", "CSP", "Battery"]

available_cols = generation_df.columns.tolist()
ordered_cols = [c for c in preferred_order if c in available_cols] + \
               [c for c in available_cols if c not in preferred_order]

generation_df = generation_df[ordered_cols]
generation_df["Total"] = generation_df.sum(axis=1).round(2)

print("\nTotal electricity generation (TWh/year) by technology\n")
display(generation_df)


#### 5.2.1. CO2 emissions from electricity generation

In [ ]:
# CO2 intensity [tCO2 per MWh_fuel] for original link carriers
co2_intensity = {
    "coal": 0.3361,
    "oil": 0.2571,
    "OCGT": 0.198,
    "CCGT": 0.198,
    "urban central gas CHP": 0.198
}

# carriers handled via Links
link_carriers = [
    "coal", "oil", "OCGT", "CCGT", "biomass", "lignite",
    "urban central solid biomass CHP", "urban central gas CHP"
]

emissions_all_years = []

for net_key, net in networks.items():
    try:
        year = int(net_key[-4:])
    except ValueError:
        continue

    # snapshots and timestep (same logic as generation function)
    snapshots_slice = slice(None)
    snapshots = net.snapshots[snapshots_slice]
    if len(snapshots) < 2:
        raise ValueError(f"Network {net_key} has insufficient snapshots")
    timestep_h = (snapshots[1] - snapshots[0]).total_seconds() / 3600

    # identify electric buses
    electric_buses = set(
        net.buses.index[
            ~net.buses.carrier.str.contains("heat|gas|H2|oil|coal", case=False, na=False)
        ]
    )

    # accumulator (tCO2)
    emissions_per_carrier = {c: 0.0 for c in co2_intensity.keys()}

    # links: take electrical output on bus1 (p1), same as generation
    for carrier in link_carriers:
        if carrier not in co2_intensity:
            continue  # we only emit for the carriers in co2_intensity

        links = net.links[
            (net.links.carrier == carrier) &
            (net.links.bus1.isin(electric_buses))
        ]
        if links.empty:
            continue

        # electrical power on bus1; keep only positive electrical output
        p1 = net.links_t.p1.loc[snapshots_slice, links.index]
        p1_positive = -p1.clip(upper=0)  # same trick: make export positive

        # Electrical energy [MWh]
        e_el_mwh = p1_positive.sum().sum() * timestep_h

        # average efficiency to back-calculate fuel input
        eta_series = links["efficiency"].replace({None: 1.0})
        # simple energy-weighted mean efficiency (robust): 
        # compute fuel per-link, then sum
        # fuel_input = sum( e_el_link / eta_link )
        # To do that, compute per-link electrical energy first:
        e_el_per_link = p1_positive.sum(axis=0) * timestep_h  # MWh per link

        # avoid division by zero / NaN
        valid = eta_series.notna() & (eta_series > 0)
        e_el_per_link = e_el_per_link[valid]
        eta_series = eta_series[valid]

        fuel_input_mwh = (e_el_per_link / eta_series).sum()

        # emissions (tCO2)
        emissions_tco2 = fuel_input_mwh * co2_intensity[carrier]
        emissions_per_carrier[carrier] += emissions_tco2

    # aggregate into Oil / Gas / Coal buckets (GtCO2)
    oil_val = emissions_per_carrier.get("oil", 0.0)
    coal_val = emissions_per_carrier.get("coal", 0.0)
    gas_val = (
        emissions_per_carrier.get("OCGT", 0.0) +
        emissions_per_carrier.get("CCGT", 0.0) +
        emissions_per_carrier.get("urban central gas CHP", 0.0)
    )

    emissions_all_years.append({
        "Year": year,
        "Oil": oil_val / 1e9,
        "Gas": gas_val / 1e9,
        "Coal": coal_val / 1e9,
        "Total": (oil_val + gas_val + coal_val) / 1e9
    })

emissions_df_gt = pd.DataFrame(emissions_all_years).set_index("Year").sort_index()

# colonna statistica (come prima)
emissions_df_gt["Total (Statistics)"] = emissions_df_gt.index.map(
    lambda y: 1.421 if y == 2023 else "-"
)

# reduction vs 2023
if 2023 in emissions_df_gt.index:
    base_2023 = emissions_df_gt.loc[2023, "Total"]
    emissions_df_gt["Reduction vs 2023 (%)"] = (
        (emissions_df_gt["Total"] - base_2023) / base_2023 * 100
    )

else:
    emissions_df_gt["Reduction vs 2023"] = float("nan")

for col in emissions_df_gt.columns:
    emissions_df_gt[col] = emissions_df_gt[col].apply(
        lambda x: f"{x:.2f}" if isinstance(x, (int, float)) and not pd.isna(x) else x
    )

display(emissions_df_gt)


Total power sector CO2 emissions source: https://www.eia.gov/environment/emissions/carbon/

#### 5.2.2. Renewable / Clean electricity generation shares

In [ ]:
eia_generation_data = pd.read_excel("./data/validation_data/annual_generation_state.xls", skiprows=1)
eia_generation_data_df = preprocess_res_ces_share_eia(eia_generation_data)[["% Actual RES", "% Actual CES"]]

In [ ]:
ces_path = project_root / "data" / "res_ces_shares" / "clean_targets.csv"
res_path = project_root / "data" / "res_ces_shares" / "res_targets.csv"

ces = pd.read_csv(ces_path, index_col=0)
res = pd.read_csv(res_path, index_col=0)

ces.columns = ces.columns.astype(str)
res.columns = res.columns.astype(str)

ces.index = ces.index.str.strip().str.upper()
res.index = res.index.str.strip().str.upper()

res_carriers = ["solar", "onwind", "offwind-ac", "offwind-dc", "ror", "hydro", "geothermal", "solar rooftop"]
ces_carriers = res_carriers + ["nuclear"]

In [ ]:
# Prepare data
res_by_network = evaluate_res_ces_by_state(
    networks, ces=ces, res=res,
    ces_carriers=ces_carriers, res_carriers=res_carriers
)

cols_per_row = 4
html_blocks = []
legend_html = """
<div style="padding:10px; margin-bottom:15px; border:1px solid #ccc; border-radius:5px; width: fit-content;">
<strong>Legend</strong>
<ul style="margin:5px 0; padding-left:20px;">
    <li style="background-color:#d4edda; padding:2px;">Diff. ≤ ±10%</li>
    <li style="background-color:#fff3cd; padding:2px;">10% &lt; Diff. ≤ ±20%</li>
    <li style="background-color:#f8d7da; padding:2px;">Diff. &gt; ±20%</li>
</ul>
</div>
"""

for network_key in sorted(networks.keys()):
    try:
        year = int(network_key[-4:])
    except ValueError:
        continue

    if year in res_by_network:
        df_year = res_by_network[year].copy()

        if year == 2023:
            df_year.drop(columns=["% RES target", "% CES target"], inplace=True, errors="ignore")
            df_year.reset_index(inplace=True)
            df_year.rename(columns={df_year.columns[0]: 'State'}, inplace=True)
            df_year = df_year.merge(eia_generation_data_df, left_on='State', right_index=True)

            df_disp = df_year.set_index('State')[['% RES', '% Actual RES', '% CES', '% Actual CES']].round(2)

            def style_row(row):
                return [
                    deviation_color(row['% RES'], row['% Actual RES']),
                    deviation_color(row['% RES'], row['% Actual RES']),
                    deviation_color(row['% CES'], row['% Actual CES']),
                    deviation_color(row['% CES'], row['% Actual CES'])
                ]

            styled_df = (
                df_disp.style
                .apply(style_row, axis=1)
                .format(fmt_2dp_or_na)
                .set_table_styles([{'selector': 'th.row_heading', 'props': 'font-weight:bold;'}])
            )
            df_html = styled_df.to_html() + legend_html  # Legend only here

        else:
            df_year = df_year[['% RES', '% RES target', '% CES', '% CES target']].round(2)
            df_year.reset_index(inplace=True)
            df_year.rename(columns={df_year.columns[0]: 'State'}, inplace=True)
            df_disp = df_year.set_index('State')[['% RES', '% RES target', '% CES', '% CES target']]

            def style_row(row):
                return [
                    simple_color(row['% RES'], row['% RES target']),
                    simple_color(row['% RES'], row['% RES target']),
                    simple_color(row['% CES'], row['% CES target']),
                    simple_color(row['% CES'], row['% CES target'])
                ]

            styled_df = (
                df_disp.style
                .apply(style_row, axis=1)
                .format(fmt_2dp_or_na)
                .set_table_styles([{'selector': 'th.row_heading', 'props': 'font-weight:bold;'}])
            )
            df_html = styled_df.to_html()

        block = f"""
        <div style="flex:1; padding:20px; min-width:300px;">
            <h4 style="text-align:left;">Year: {year}</h4>
            {df_html}
        </div>
        """
        html_blocks.append(block)

rows = [
    "<div style='display:flex; gap:10px; flex-wrap:wrap;'>" +
    "".join(html_blocks[i:i+cols_per_row]) +
    "</div>"
    for i in range(0, len(html_blocks), cols_per_row)
]

for row in rows:
    display(HTML(row))


In [ ]:
display_grid_region_results(
    networks=networks,
    ces=ces,
    res=res,
    ces_carriers=ces_carriers,
    res_carriers=res_carriers
)


### 5.3. Hydrogen dispatch

*Similar to electricity dispatch, the dispatch of hydrogen through the model years is assessed with detail on the technologies used for its production.*

In [ ]:
h2_carriers = [
    "Alkaline electrolyzer large",
    "Alkaline electrolyzer medium",
    "Alkaline electrolyzer small",
    "PEM electrolyzer",
    "SOEC"
]

In [ ]:
hydrogen_dispatch_df = plot_hydrogen_dispatch(networks, h2_carriers)

### 5.3.1. Hourly matching

In [ ]:
results_subplots = analyze_additionality_multiple_subplots(
    networks=networks,
    regions=None,  # Auto-detect regions, or pass explicit list
    include_whole_country=False,  # Add whole country as a subplot
    nice_names_power=nice_names_power,
    nice_names=nice_names,
    tech_power_color=tech_power_color,
    tech_colors=tech_colors,
    skip_years=[2023],
    show_plots=True,
    show_scenario_name=False,
    ncols=2,  # Number of columns in subplot grid
    subplot_height=4,  # Height of each subplot
    subplot_width=9,   # Width of each subplot
    shared_ylim=False,
    min_electrolyzer_mw=1e-3,
    # start_date='01-01',
    # end_date='01-08',
)

### 5.3.2. Hydrogen VS e-kerosene production

In [ ]:
hydrogen_ekerosene_prod = {}
plot_threshold = 1e-3  # GW threshold, same as in capacity factor filtering

print(f"\nApplying production filter: skipping networks with daily-average production < {plot_threshold:.3e} GW ({plot_threshold*1e3:.1f} MW)\n")

for name, net in networks.items():
    year = name[-4:]
    hydrogen_ekerosene_prod[name] = compare_h2_kerosene_production(
        net,
        network_name=year,
        plot_threshold_gw=plot_threshold
    )

In [ ]:
h2_capacity_factors = {}

print("\nCapacity factor for H2 production\n")
for name, net in networks.items():
    year = name[-4:]

    # compute CF for this network
    results = compute_capacity_factor_electrolysis({year: net})
    h2_capacity_factors[name] = results

    for y, output in results.items():
        df_regions = output["regions"]
        df_links = output["links"]

        # filter regions with negligible capacity
        threshold_gw = 1e-6  # 1 MW
        print(f"\nApplying filter: excluding regions with installed capacity < {threshold_gw:.3e} GW ({threshold_gw*1e3:.1f} MW)")
        
        df_filtered = df_regions[df_regions["Capacity (GW input electricity)"] >= threshold_gw].copy()
        if df_filtered.empty:
            continue

        df_filtered.reset_index(drop=True, inplace=True)

        print(f"\nYear: {year}\n")
        display(
            df_filtered.style
            .hide(axis="index")
            .format({
                "Capacity (GW input electricity)": "{:.2f}",
                "Electricity input (MWh)": "{:.0f}",
                "Hydrogen output (MWh)": "{:.0f}",
                "Capacity factor (%)": "{:.2f}",
            })
        )

#### Definition: Capacity factor

The capacity factor of electrolyzers is calculated as:

$$
\text{Capacity Factor (\%)} =
\frac{\text{Electricity input (MWh)}}{\text{Installed capacity (MW)} \times \text{Total weighted hours}} \times 100
$$


---

## 6. Economic Analysis

*This section shows the evolution of levelized costs of electricity and hydrogen throughout the model time horizon.*

### 6.1. Marginal prices of electricity

In [ ]:
for key, net in networks.items():
    region_prices = extract_marginal_price_by_grid_region_weighted(net)
    plot_marginal_prices_by_region_weighted(
        region_prices,
        network=net,
        network_name=key,
        demand_threshold=0.01
    )

### 6.2. Levelized Cost of Electricity (LCOE)
*The LCOE is assessed at the level of the different US grid regions.*

In [ ]:
grid_regions_shapes = gpd.read_file(grid_region_shapes_path)

In [ ]:
# Collect LCOE values and cache results
all_weighted_lcoe = []
cached_results = {}

for key, net in networks.items():
    lcoe_gdf, table, lcoe_by_bus, lcoe_data, *_ = calculate_lcoe_summary_and_map(net, grid_regions_shapes)
    cached_results[key] = (net, lcoe_gdf, table, lcoe_by_bus, lcoe_data)

    # Map EMM regions and transmission fees
    # emm_mapping = dict(zip(net.buses.grid_region, net.buses.emm_region))
    table["EMM Region"] = table["grid_region"].map(emm_mapping)

    fee_map = regional_fees.loc[
        regional_fees["Year"] == int(key.split("_")[-1]), ["region", "Transmission nom USD/MWh", "Distribution nom USD/MWh"]
    ].set_index("region")[["Transmission nom USD/MWh", "Distribution nom USD/MWh"]]

    table["Transmission (USD/MWh)"] = table["EMM Region"].map(fee_map.loc[:,"Transmission nom USD/MWh"])
    table["Distribution (USD/MWh)"] = table["EMM Region"].map(fee_map.loc[:,"Distribution nom USD/MWh"])
    table.drop(columns=["EMM Region"], inplace=True)

    table.loc[:,"LCOE_transmission_fees (USD/MWh)"] = table.loc[:,"Weighted Average LCOE (USD/MWh)"] + table.loc[:, "Transmission (USD/MWh)"]
    table.loc[:,"LCOE_transmission_distribution_fees (USD/MWh)"] =  table.loc[:,"LCOE_transmission_fees (USD/MWh)"] + table.loc[:, "Distribution (USD/MWh)"]

    merged = lcoe_by_bus.merge(lcoe_data[['bus', 'energy']], on='bus', how='left')
    grouped = merged.groupby('grid_region').apply(
        lambda df: (df['weighted_lcoe'] * df['energy']).sum() / df['energy'].sum()
    )
    all_weighted_lcoe.extend(grouped.dropna().values)

# Compute global vmin/vmax
vmin = np.quantile(all_weighted_lcoe, 0.05)
vmax = np.quantile(all_weighted_lcoe, 0.95)

# Plot using cached data
plots_data = {}

for key, (net, lcoe_gdf, table, lcoe_by_bus, lcoe_data) in cached_results.items():
    year_match = re.search(r'\d{4}', key)
    year_str = year_match.group() if year_match else "Year N/A"
    title = f"Weighted average (plant level) LCOE per Grid Region (USD/MWh) - {year_str}"

    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw={'projection': ccrs.PlateCarree()})
    plots_data[key] = table

    plot_lcoe_map_by_grid_region(
        lcoe_by_bus, lcoe_data, grid_regions_shapes,
        title=title, key=key, ax=ax,
        vmin=vmin, vmax=vmax
    )

    showfig()


In [ ]:
def uppercase_only_lcoe_word(df):
    new_cols = {
        col: re.sub(r'\blcoe\b', 'LCOE', col, flags=re.IGNORECASE)
        for col in df.columns
    }
    return df.rename(columns=new_cols)

def apply_nice_names_to_columns_custom(df, mapping):
    sorted_mapping = sorted(mapping.items(), key=lambda item: -len(item[0]))

    placeholder_prefix = "__PLACEHOLDER__"
    placeholder_map = {}
    
    new_cols = []
    for col in df.columns:
        temp_col = col
        for i, (key, val) in enumerate(sorted_mapping):
            if key in temp_col:
                placeholder = f"{placeholder_prefix}{i}__"
                placeholder_map[placeholder] = val
                temp_col = temp_col.replace(key, placeholder)
        new_cols.append(temp_col)
    
    final_cols = [
        reduce(lambda c, p: c.replace(p[0], p[1]), placeholder_map.items(), col)
        for col in new_cols
    ]
    
    df.columns = final_cols
    return df

for net_name, table in plots_data.items():
    year = net_name[-4:]
    print(f"\nYear: {year}")

    if table.index.name == "grid_region":
        table = table.reset_index()

    table = table.rename(columns={"grid_region": "Grid Region"})
    table = table.rename(columns={"Transmission (USD/MWh)": "Transmission fees (USD/MWh)"})
    table = table.rename(columns={"Distribution (USD/MWh)": "Distribution fees (USD/MWh)"})
    table = table.rename(columns={"LCOE_transmission_fees (USD/MWh)": "LCOE + Transmission fees (USD/MWh)"})
    table = table.rename(columns={"LCOE_transmission_distribution_fees (USD/MWh)": "LCOE + T&D fees (USD/MWh)"})

    table.index = pd.Index([""] * len(table))

    table = uppercase_only_lcoe_word(table)
    table = apply_nice_names_to_columns_custom(table, nice_names)

    display(
        table.style.format(precision=2, thousands=",")
    )

In [ ]:
for key, net in networks.items():
    _, _, _, lcoe_data, *_ = calculate_lcoe_summary_and_map(net, grid_regions_shapes)

    region_summary = lcoe_data.groupby(['grid_region', 'bus', 'carrier']).agg(
            dispatch_mwh=('energy', 'sum'),
            total_cost=('lcoe', lambda x: (x * lcoe_data.loc[x.index, 'energy']).sum())
        ).reset_index()

    region_summary['lcoe'] = region_summary['total_cost'] / region_summary['dispatch_mwh']
    region_summary['dispatch'] = region_summary['dispatch_mwh'] / 1e6
    region_summary['bus'] = region_summary['bus'].str.extract(r'(US\d{1} \d{1,2})')

    region_bus_summary = region_summary.pivot(index=['grid_region', 'bus'], columns='carrier', values=['lcoe', 'dispatch'])
    region_bus_summary.columns = [
        f"{carrier} {metric} ({'USD/MWh' if metric == 'lcoe' else 'TWh'})"
        for metric, carrier in region_bus_summary.columns
    ]
    region_bus_summary = region_bus_summary.reset_index()

    dispatch_cols = [col for col in region_bus_summary.columns if 'dispatch' in col.lower()]
    for col in dispatch_cols:
        region_bus_summary[col] = pd.to_numeric(region_bus_summary[col], errors='coerce').fillna(0.0)

    lcoe_cols = [col for col in region_bus_summary.columns if 'lcoe' in col.lower()]

    min_dispatch_threshold = 1  # TWh
    for lcoe_col in lcoe_cols:
        carrier = lcoe_col.split(" ")[0]
        dispatch_col = next((col for col in dispatch_cols if col.startswith(carrier + " ")), None)
        if dispatch_col:
            mask = region_bus_summary[dispatch_col] < min_dispatch_threshold
            region_bus_summary.loc[mask, lcoe_col] = np.nan
    
    table_df = region_bus_summary.groupby(['grid_region']).agg(['min', 'max']).drop(columns=['bus'])

    plot_float_bar_lcoe_dispatch_ranges(table_df, key, nice_names)


In [ ]:
for key, net in networks.items():
    # Recalculate lcoe_data and region_bus_summary
    _, _, _, lcoe_data, *_ = calculate_lcoe_summary_and_map(net, grid_regions_shapes)

    region_summary = lcoe_data.groupby(['grid_region', 'bus', 'carrier']).agg(
        dispatch_mwh=('energy', 'sum'),
        total_cost=('lcoe', lambda x: (x * lcoe_data.loc[x.index, 'energy']).sum())
    ).reset_index()

    region_summary['lcoe'] = region_summary['total_cost'] / region_summary['dispatch_mwh']
    region_summary['dispatch'] = region_summary['dispatch_mwh'] / 1e6
    region_summary['bus'] = region_summary['bus'].str.extract(r'(US\d{1} \d{1,2})')

    region_bus_summary = region_summary.pivot(index=['grid_region', 'bus'], columns='carrier', values=['lcoe', 'dispatch'])
    region_bus_summary.columns = [
        f"{carrier} {metric} ({'USD/MWh' if metric == 'lcoe' else 'TWh'})"
        for metric, carrier in region_bus_summary.columns
    ]
    region_bus_summary = region_bus_summary.reset_index()

    # Convert dispatch to numeric and apply threshold filtering to LCOE
    dispatch_cols = [col for col in region_bus_summary.columns if 'dispatch' in col.lower()]
    lcoe_cols = [col for col in region_bus_summary.columns if 'lcoe' in col.lower()]
    
    for col in dispatch_cols:
        region_bus_summary[col] = pd.to_numeric(region_bus_summary[col], errors='coerce').fillna(0.0)

    min_dispatch_threshold = 1  # TWh
    for lcoe_col in lcoe_cols:
        carrier = lcoe_col.split(" ")[0]
        dispatch_col = next((col for col in dispatch_cols if col.startswith(carrier + " ")), None)
        if dispatch_col:
            mask = region_bus_summary[dispatch_col] < min_dispatch_threshold
            region_bus_summary.loc[mask, lcoe_col] = np.nan

    # --- NATIONAL PLOT ONLY ---
    national_data = region_bus_summary.drop(columns=['grid_region', 'bus'])
    lcoe_columns = [col for col in national_data.columns if 'lcoe' in col.lower()]
    national_data = national_data[lcoe_columns]

    national_min = national_data.min()
    national_max = national_data.max()

    arrays = [[col for col in national_min.index] * 2, ['min'] * len(national_min) + ['max'] * len(national_max)]
    multi_cols = pd.MultiIndex.from_tuples(zip(*arrays))

    national_values = pd.concat([national_min, national_max])
    national_df = pd.DataFrame([national_values.values], columns=multi_cols, index=['USA'])

    display(Markdown("\n**Note:** The LCOE values reported here also include existing nuclear plants, many of which are already amortized and were built under different historical cost conditions. In contrast, Lazard’s estimates reflect newly built projects with current capital costs and construction standards, which explains the higher LCOE values."))
    plot_float_bar_lcoe_dispatch_ranges(national_df, key, nice_names)


### 6.3. Levelized Cost of Hydrogen (LCOH)
*The levelized cost of hydrogen is assessed at the level of the different US Grid Regions.*

In [ ]:
## Using Marginal price as electricity price
plot_lcoh_maps_by_grid_region_marginal(
    networks,
    grid_regions_shapes,
    h2_carriers,
    regional_fees,
    emm_mapping,
    output_threshold=0
)

In [ ]:
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, grid_regions_shapes
)

# Using LCOE as electricity cost
plot_lcoh_maps_by_grid_region_lcoe(
    networks=networks,
    shapes=grid_regions_shapes,
    h2_carriers=h2_carriers,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    output_threshold=0
)


In [ ]:
# LCOH using marginal price as electricity price
res_marginal = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    electricity_price="marginal",
    include_baseload=True,  # Include baseload charges
    baseload_charge_path="./data/energy_charge_rate.csv",
    customer_charge_mw=400,  # in MW
    demand_charge_rate=9.0   # in $/kW/month
)

print("\nLCOH using marginal price as electricity price")
for year, df in res_marginal.items():
    print(f"\nYear {year}:")

    # format only numeric columns
    numeric_cols = df.select_dtypes(include="number").columns
    fmt = {col: "{:.2f}" for col in numeric_cols}

    display(df.style.format(fmt).hide(axis="index"))


In [ ]:
# Compute LCOE per bus and per region
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, regions_onshore
)

# Weighted average LCOE per grid region (USD/MWh el)
grid_region_weighted_lcoe = (
    lcoe_by_bus.merge(lcoe_data[['bus', 'energy']], on='bus', how='left')
    .groupby('grid_region')
    .apply(lambda df: (df['weighted_lcoe'] * df['energy']).sum() / df['energy'].sum())
)

# LCOH using LCOE as electricity price
res_lcoe = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    electricity_price="LCOE",
    grid_region_lcoe=grid_region_weighted_lcoe,
    include_baseload=True,  # Include baseload charges
    baseload_charge_path="./data/energy_charge_rate.csv",
    customer_charge_mw=400,  # in MW
    demand_charge_rate=9.0   # in $/kW/month
)

print("\nLCOH using LCOE as electricity price")
for year, df in res_lcoe.items():
    print(f"\nYear {year}:\n")

    # format only numeric columns
    numeric_cols = df.select_dtypes(include="number").columns
    fmt = {col: "{:.2f}" for col in numeric_cols}

    display(df.style.format(fmt).hide(axis="index"))


---

## 7. E-Kerosene

*This section is dedicated to the analysis of e-kerosene production and costs.*

## 7.1. Aviation fuel demand

In [ ]:
df = compute_aviation_fuel_demand(networks)

styled_df = (
    df.style
    .hide(axis="index")
    .format({
        "Kerosene (TWh)": "{:,.2f}",
        "e-Kerosene (TWh)": "{:,.2f}",
        "Total (TWh)": "{:,.2f}",
        "e-Kerosene Share (%)": "{:.2f}%",
    })
    .set_properties(**{"text-align": "right", "font-size": "13px"})
    .set_table_styles([
        {"selector": "th", "props": [("text-align", "right"), ("font-size", "12px")]},
        {"selector": "caption", "props": [("caption-side", "top"), ("font-weight", "bold"), ("font-size", "16px")]}
    ])
)

print("\nAviation fuel demand\n")
display(styled_df)

In [ ]:
passenger_data = pd.read_csv("./data/T100_Domestic_Market_and_Segment_Data_-3591723781169319541.csv")
total_passengers = passenger_data.passengers.sum()

In [ ]:
for network_name, n in networks.items():
    print(f"\nYear: {network_name[-4:]}\n")
    ekerosene_demand_state = compute_aviation_demand_table(n)
    ekerosene_demand_state.loc[:, "n. of Passengers"] = (
        ekerosene_demand_state["Share (%)"] * total_passengers / 100)
    display(ekerosene_demand_state.set_index("State")[["Fuel demand (TWh)", "n. of Passengers", "Share (%)"]])

In [ ]:
for network_name, n in networks.items():    
    print(f"\nYear: {network_name[-4:]}\n")
    create_aviation_demand_by_state_map(
        n,
        grid_region_shapes_path,
        network_name=network_name,
        min_demand_twh=1.0,
        year_title=True
    )
    showfig()

In [ ]:
for network_name, n in networks.items():
    print(f"\nYear: {network_name[-4:]}\n")
    ekerosene_demand_state = compute_aviation_demand_table(n, level="grid_region")
    ekerosene_demand_state.loc[:, "n. of Passengers"] = (
        ekerosene_demand_state["Share (%)"] * total_passengers / 100)
    display(ekerosene_demand_state.set_index("Grid Region")[["Fuel demand (TWh)", "n. of Passengers", "Share (%)"]])


In [ ]:
for network_name, n in networks.items():    
    print(f"\nYear: {network_name[-4:]}\n")
    create_aviation_demand_by_grid_region_map(
        n,
        grid_region_shapes_path,
        network_name=network_name,
        min_demand_twh=1.0,
        year_title=True
    )
    showfig()

In [ ]:
results_state = {}

for name, net in networks.items():
    year = name[-4:]
    print(f"\nYear: {year}")
    table = compute_aviation_shares(net, level="state")
    results_state[name] = table
    display(table)

In [ ]:
# Grid regions 

results_grid = {}

for name, net in networks.items():
    year = name[-4:]
    print(f"\nYear: {year}")
    table = compute_aviation_shares(net, level="grid_region")
    results_grid[name] = table
    display(table)

In [ ]:
## 7.2. Required inputs for e-kerosene production

In [ ]:
# Run calculation
ft_df = calculate_total_inputs_outputs_ft(networks)

print("\nElectricity / hydrogen / CO2 balance for e-kerosene production \n")

# Display without index
display(ft_df.style.hide(axis="index").format(precision=2))


## 7.2. Capacity factor of e-kerosene production plants

In [ ]:
# Capacity factor FT plants
cf = compute_ft_capacity_factor(networks)

print("\nFT plants capacity factor")
for year, df in cf.items():
    print(f"\nYear: {year}\n")

    # format all numeric columns to 2 decimals
    numeric_cols = df.select_dtypes(include="number").columns
    fmt = {col: "{:.2f}" for col in numeric_cols}

    display(df.style.format(fmt).hide(axis="index"))

---

## 7.3. Economic analysis

### 7.3.1. Levelized cost of CO2 (LCOC)

In [ ]:
print("LCOC computed using marginal price as electricity price")

lcoc_by_region = compute_LCOC_by_region(
    networks=networks,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    electricity_price="marginal",
    year_title=True,
    captured_threshold_mt=1e-3
)

In [ ]:
# LCOE per grid region
lcoe_gdf, table, lcoe_by_bus, lcoe_data, vmin, vmax = calculate_lcoe_summary_and_map(
    n, regions_onshore
)
lcoe_by_region = (
    lcoe_by_bus.merge(lcoe_data[['bus', 'energy']], on='bus', how='left')
    .groupby('grid_region')
    .apply(lambda df: (df['weighted_lcoe'] * df['energy']).sum() / df['energy'].sum())
)

# LCOH per grid region (USD/kg H2)
lcoh_by_region = calculate_lcoh_by_region(
    networks,
    h2_carriers,
    regional_fees,
    emm_mapping,
    electricity_price="marginal",
    include_baseload=True,
)

In [ ]:
print("LCOC computed using LCOE as electricity price")

lcoc_by_region = compute_LCOC_by_region(
    networks=networks,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    electricity_price="lcoe",
    lcoe_by_region=lcoe_by_region,
    year_title=True,
    captured_threshold_mt=1e-2
)

### 7.3.2. Levelized cost of e-kerosene

In [ ]:
# LCO e-kerosene using marginal prices

print("LCO e-kerosene computed using marginal prices for electricity, hydrogen, and CO2")

LCO_ekerosene_by_region = compute_LCO_ekerosene_by_region(
    networks=networks,
    fx_2020=1.14,
    fx_recent=1.08,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    unit="gal",
    year_title=True,
    p_nom_threshold=1,   # MW
    electricity_price="marginal",
    hydrogen_price="marginal",
    co2_price="lcoc",
    lcoc_by_region=lcoc_by_region,
)

In [ ]:
lcoc_by_region = calculate_LCOC_by_region(
    networks=networks,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    electricity_price="marginal",
    captured_threshold_mt=1e-3
)

In [ ]:
print("LCO e-kerosene computed using LCOE, LCOH (LCOH + Transmission fees + Baseload charges computed using marginal price of electricity) and LCOC (computed using marginal price of electricity) as input prices")

LCO_ekerosene_by_region = compute_LCO_ekerosene_by_region(
    networks=networks,
    fx_2020=1.14,
    fx_recent=1.08,
    regional_fees=regional_fees,
    emm_mapping=emm_mapping,
    unit="gal",           
    year_title=True,
    p_nom_threshold=1, # MW
    electricity_price="lcoe",
    hydrogen_price="lcoh",
    co2_price="lcoc",
    lcoe_by_region=lcoe_by_region,
    lcoh_by_region=lcoh_by_region,
    lcoc_by_region=lcoc_by_region
)

#### Definition: Levelized Cost of e-Kerosene

The levelized cost of e-kerosene is calculated as:

$$
\text{LCO}_{\text{e-kerosene}} \,[\$/\text{gal}] =
\frac{\sum_t \big(P^{\text{elec}}_t \cdot E^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{\sum_t \big(P^{\text{H2}}_t \cdot H^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{\sum_t \big(P^{\text{CO2}}_t \cdot C^{\text{in}}_t\big)}{\sum_t E^{\text{out}}_t}
+ \frac{CAPEX \cdot P_{\text{nom}}}{\sum_t E^{\text{out}}_t}
+ VOM
$$

---

## Definitions

- $P^{\text{elec}}_t$ = electricity price at time *t* (USD/MWh)
- $E^{\text{in}}_t$ = electricity input at time *t* (MWh)

- $P^{\text{H2}}_t$ = hydrogen price at time *t* (USD/MWh)
- $H^{\text{in}}_t$ = hydrogen input at time *t* (MWh)

- $P^{\text{CO2}}_t$ = CO2 price at time *t* (USD/t CO2)
- $C^{\text{in}}_t$ = CO2 input at time *t* (t CO2)

- $E^{\text{out}}_t$ = e-kerosene output at time *t* (MWh)

- $CAPEX$ = annualized capital cost (USD/MW a)
- $P_{\text{nom}}$ = installed Fischer–Tropsch plant capacity (MW)

- $VOM$ = variable O&M cost (USD/MWh)

- $\eta_{\text{gal}}$ = conversion factor from MWh to gallons  

$$
\eta_{\text{gal}} =
\frac{34 \,\text{MJ/l} \times 3.78541 \,\text{L/gal}}{3600 \,\text{MJ/MWh}}
\;\approx\; 0.0357 \,\text{MWh/gal}
$$


---

## 8. Industrial sector analysis

## 8.1 Industrial production
This section focuses on the production from the different industrial sectors related to e-kerosene production (point-source CO2).

In [ ]:
carriers_of_interest = ['NH3', 'ethanol', 'DRI', 'steel BF-BOF', 'steel EAF', 'cement']

In [ ]:
# Specific conversion per carrier
conversion_factors = {
    "cement": (1 / 1000, "kt/year"),
    "DRI": (1 / 1000, "kt/year"),
    "steel BF-BOF": (1 / 1000, "kt/year"),
    "steel EAF": (1 / 1000, "kt/year"),
    "ethanol": (1/ 80.2 * 3600 / 1e6, "MMgallon/year"),
    "NH3": (1/ 5.17 / 1e3, "thousand metric tons/year"), 
}

summary_all = {}
units = {}

for name, network in networks.items(): 

    filtered_loads = network.loads[network.loads["carrier"].isin(carriers_of_interest)].copy()

    filtered_loads["state"] = filtered_loads["bus"].map(network.buses["state"])

    summary = (
        filtered_loads.groupby("carrier")["p_set"]
        .sum()
        .mul(8760)  # convert from MW to MWh/year
        .reindex(carriers_of_interest)
        .fillna(0)
    )

    converted = []
    unit_column = []
    for carrier in carriers_of_interest:
        value = summary.get(carrier, 0)
        factor, unit = conversion_factors.get(carrier, (1, "MWh/year"))
        converted.append(round(value * factor, 2))
        unit_column.append(unit)

    summary_converted = pd.Series(converted, index=carriers_of_interest)
    year = name[-4:]  # Extract year from network name
    summary_all[year] = summary_converted

    if "Unit" not in units:
        units["Unit"] = unit_column 

summary_df = pd.DataFrame(summary_all)
summary_df["Unit"] = units["Unit"]

pd.set_option('display.float_format', '{:,.2f}'.format)

print("\nTotal production by industrial product\n")
display(summary_df)

## 8.2 Industrial emissions
This section focuses on the analysis of the CO2 commodity, with particular reference to the industrial sector and the use for e-kerosene production.

In [ ]:
for key, net in networks.items():
    year = key[-4:]
    print(f"\nYear: {year}\n")

    df = compute_emissions_from_links(net)

    df_nonzero = df[
        (df["CO2 to Atmosphere (Mt CO2/year)"] != 0) |
        (df["CO2 Captured (Mt CO2/year)"] != 0) |
        (df["CO2 Sequestered (Mt CO2/year)"] != 0) |
        (df["Net CO2 Emissions (Mt CO2/year)"] != 0)
    ]

    display(df_nonzero.style.format(precision=2).hide(axis="index"))


In [ ]:
carrier_groups = {
    "BF-BOF": ["BF-BOF", "BF-BOF CC"],
    "DRI": ["DRI", "DRI CC"],
    "ethanol": ["ethanol from starch", "ethanol from starch CC"],
    "SMR": ["SMR", "SMR CC"],
    "dry clinker": ["dry clinker", "dry clinker CC"],
    "DAC": ["DAC"]
}

In [ ]:
for key, net in networks.items():
    
    year = key[-4:]
    print(f"\nYear: {year}\n")

    # emissions summary
    df_summary = compute_emissions_grouped(net, carrier_groups)

    # Remove sequestered column entirely (do not use it downstream)
    if "CO2 Sequestered (Mt CO2/year)" in df_summary.columns:
        df_summary = df_summary.drop(columns=["CO2 Sequestered (Mt CO2/year)"])

    # Keep non-zero rows only (updated to exclude removed column)
    filter_cols = [
        "CO2 to Atmosphere (Mt CO2/year)",
        "CO2 Captured (Mt CO2/year)",
        "Net CO2 Emissions (Mt CO2/year)"
    ]

    df_filtered = df_summary[
        (df_summary[filter_cols].abs().sum(axis=1) > 0)
    ].copy()

    # Clean numerical noise
    for col in filter_cols:
        df_filtered[col] = df_filtered[col].apply(lambda x: 0.0 if abs(x) < 1e-6 else x)

    # Biogenic-only scenarios
    biogenic_only = any(s in key for s in [
        "scenario_05", "scenario_06"
    ])
    if biogenic_only:
        # If ethanol sequestration existed, it's now removed from df_summary,
        # but you still want to compute biogenic stored.
        # Pull it from net instead of df_filtered.
        # You did not ask to change this part, so keep same logic:
        ethanol_mask = df_summary["carrier group"] == "ethanol"
        total_stored = 0.0  # now sequestration column is removed
        note = " (biogenic CO2 only)"
    else:
        total_stored = 0.0
        note = ""

    # Geological storage accounting remains unchanged
    co2_stores = net.stores.index[net.stores.carrier == "co2 stored"]

    if len(co2_stores):
        w = net.snapshot_weightings["stores"]

        inflow_Mt = (
            (-net.stores_t.p[co2_stores].clip(upper=0))
            .mul(w, axis=0).sum().sum() / 1e6
        )
        start_stock_Mt = net.stores.loc[co2_stores, "e_initial"].fillna(0).sum() / 1e6
        end_stock_Mt   = net.stores_t.e.loc[net.snapshots[-1], co2_stores].sum() / 1e6
    else:
        inflow_Mt = start_stock_Mt = end_stock_Mt = 0.0

    # Print summary
    print(f"Total CO2 sequestered{note} (annual flows): {total_stored:.2f} Mt CO2")
    print(f"Total CO2 injected into geological storage - annual inflow: {inflow_Mt:.2f} Mt CO2")
    print(f"Start stock: {start_stock_Mt:.2f} Mt CO2")
    print(f"End stock: {end_stock_Mt:.2f} Mt CO2\n")

    # Show table only if not empty
    if not df_filtered.empty:
        display(df_filtered.style.format(precision=2).hide(axis='index'))
    else:
        print("")


In [ ]:
all_dfs = []
global_max = 0.0

for name, net in networks.items():
    year = re.search(r"\d{4}", str(name)).group()
    df_state = compute_emissions_by_state(net, carrier_groups)
    if df_state.empty:
        continue

    df_state["year"] = year
    all_dfs.append(df_state)

    mx = df_state["Net CO2 Emissions (Mt CO2/year)"].abs().max()
    if mx > global_max:
        global_max = mx

for df in all_dfs:
    year = df["year"].iloc[0]
    plot_emissions_maps_by_group(
        all_state_emissions=df,
        path_shapes=state_shapes_path,
        title=year,
        column="Net CO2 Emissions (Mt CO2/year)",
        vmin=-global_max,
        vmax= global_max
    )
